In [1]:
from google.colab import files
from google.colab import drive

In [2]:
#Package Installation
!pip3 install presto --user

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 584.8/584.8 kB 8.4 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 55.3 MB/s eta 0:00:00
  Created wheel for presto: filename=presto-0.7.2-py3-none-any.whl size=119350 sha256=b9f6836cd01720a5cb7fa761759d235ad1bc4e663013ac0320b1422cc662893c
  Stored in directory: /root/.cache/pip/wheels/2b/2e/ac/9e2103265126c4d42298796fc1f0b1946d32be9d2efd35afaa
Successfully built presto


In [38]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
#Add files
drive.mount('/content/drive')

Mounted at /content/drive


In [39]:
uploaded = files.upload()

Saving 5_S3_L001_R2_001.fastq to 5_S3_L001_R2_001.fastq
Saving AlpVHH_R2_primers_05Feb24.fasta to AlpVHH_R2_primers_05Feb24 (1).fasta
Saving 5_S3_L001_R1_001.fastq to 5_S3_L001_R1_001.fastq
Saving AlpVHH_R1_primers_05Feb24.fasta to AlpVHH_R1_primers_05Feb24 (1).fasta


### **Paired-End Assembly**

In [40]:
!/usr/bin/python3 /root/.local/bin/AssemblePairs.py align -1 5_S3_L001_R1_001.fastq -2 5_S3_L001_R2_001.fastq \
    --coord sra --rc tail --outname M1 --log AP.log

       START> AssemblePairs
     COMMAND> align
       FILE1> 5_S3_L001_R1_001.fastq
       FILE2> 5_S3_L001_R2_001.fastq
  COORD_TYPE> sra
       ALPHA> 1e-05
   MAX_ERROR> 0.3
     MIN_LEN> 8
     MAX_LEN> 1000
SCAN_REVERSE> False
       NPROC> 2

PROGRESS> 19:55:00 |####################| 100% (159,502) 57.3 min

OUTPUT> M1_assemble-pass.fastq
 PAIRS> 159502
  PASS> 138377
  FAIL> 21125
   END> AssemblePairs



In [41]:
!/usr/bin/python3 /root/.local/bin/ParseLog.py -l AP.log -f ID LENGTH OVERLAP ERROR PVALUE

START> ParseLog
 FILE> AP.log

PROGRESS> 19:55:05 (159502) 0.1 min

 OUTPUT> AP_table.tab
RECORDS> 159502
   PASS> 159502
   FAIL> 0
    END> ParseLog



In [42]:
!cat AP_table.tab
#Might crash notebook

Streaming output truncated to the last 5000 lines.
M02058:623:000000000-DMLF3:1:1101:7668:13752	381	121	0.0826	-5.2857e-55
M02058:623:000000000-DMLF3:1:1101:26102:13753	414	88	0.0000	-5.2202e-54
M02058:623:000000000-DMLF3:1:1101:22432:13753	414	88	0.0000	-5.2202e-54
M02058:623:000000000-DMLF3:1:1101:10659:13753	396	106	0.0094	-2.4157e-62
M02058:623:000000000-DMLF3:1:1101:24691:13753	426	76	0.0395	-1.6624e-40
M02058:623:000000000-DMLF3:1:1101:24977:13753	251	251	0.0159	-4.9940e-142
M02058:623:000000000-DMLF3:1:1101:18404:13754	397	105	0.0000	-3.0386e-64
M02058:623:000000000-DMLF3:1:1101:28416:13754	384	118	0.0000	-4.5278e-72
M02058:623:000000000-DMLF3:1:1101:23931:13754	428	74	0.6081	3.6671e-03
M02058:623:000000000-DMLF3:1:1101:24470:13754	420	82	0.0610	-1.4177e-40
M02058:623:000000000-DMLF3:1:1101:12170:13754	418	84	0.0000	-1.3364e-51
M02058:623:000000000-DMLF3:1:1101:18556:13754	420	82	0.0122	-5.2600e-48
M02058:623:000000000-DMLF3:1:1101:26773:13756	426	76	0.1053	-1.0835e-32
M02058:62

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



### **Quality Control Filtering**

In [6]:
!/usr/bin/python3 /root/.local/bin/FilterSeq.py quality -s M1_assemble-pass.fastq -q 30 --outname M1 --log FS.log

   START> FilterSeq
 COMMAND> quality
    FILE> M1_assemble-pass.fastq
   INNER> False
MIN_QUAL> 30.0
   NPROC> 2

PROGRESS> 17:02:32 |####################| 100% (138,377) 2.9 min

   OUTPUT> M1_quality-pass.fastq
SEQUENCES> 138377
     PASS> 35005
     FAIL> 103372
      END> FilterSeq



In [8]:
!/usr/bin/python3 /root/.local/bin/ParseLog.py -l FS.log -f ID QUALITY

START> ParseLog
 FILE> FS.log

PROGRESS> 17:03:18 (138377) 0.0 min

 OUTPUT> FS_table.tab
RECORDS> 138377
   PASS> 138377
   FAIL> 0
    END> ParseLog



In [ ]:
!cat FS_table.tab

### **Masking Primers**

In [14]:
!/usr/bin/python3 /root/.local/bin/MaskPrimers.py score -s M1_quality-pass.fastq -p AlpVHH_R1_primers_05Feb24.fasta \
   --mode mask --pf R1Primer --outname M1-FWD --log MPV.log
!/usr/bin/python3 /root/.local/bin/MaskPrimers.py score -s M1-FWD_primers-pass.fastq -p AlpVHH_R2_primers_05Feb24.fasta \
   --mode cut --revpr --pf R2Primer --outname M1-REV --log MPC.log

       START> MaskPrimers
     COMMAND> score
    SEQ_FILE> M1_quality-pass.fastq
 PRIMER_FILE> AlpVHH_R1_primers_05Feb24.fasta
        MODE> mask
   MAX_ERROR> 0.2
   START_POS> 0
  REV_PRIMER> False
     BARCODE> False
PRIMER_FIELD> R1Primer
       NPROC> 2

PROGRESS> 17:21:04 |####################| 100% (35,005) 0.7 min

   OUTPUT> M1-FWD_primers-pass.fastq
SEQUENCES> 35005
     PASS> 34743
     FAIL> 262
      END> MaskPrimers

       START> MaskPrimers
     COMMAND> score
    SEQ_FILE> M1-FWD_primers-pass.fastq
 PRIMER_FILE> AlpVHH_R2_primers_05Feb24.fasta
        MODE> cut
   MAX_ERROR> 0.2
   START_POS> 0
  REV_PRIMER> True
     BARCODE> False
PRIMER_FIELD> R2Primer
       NPROC> 2

PROGRESS> 17:21:54 |####################| 100% (34,743) 0.8 min

   OUTPUT> M1-REV_primers-pass.fastq
SEQUENCES> 34743
     PASS> 34709
     FAIL> 34
      END> MaskPrimers



In [15]:
!/usr/bin/python3 /root/.local/bin/ParseLog.py -l MPV.log MPC.log -f ID PRIMER ERROR

START> ParseLog
 FILE> MPV.log

PROGRESS> 17:24:57 (35005) 0.0 min

 OUTPUT> MPV_table.tab
RECORDS> 35005
   PASS> 35005
   FAIL> 0
    END> ParseLog

START> ParseLog
 FILE> MPC.log

PROGRESS> 17:24:58 (34743) 0.0 min

 OUTPUT> MPC_table.tab
RECORDS> 34743
   PASS> 34743
   FAIL> 0
    END> ParseLog



In [ ]:
!cat MPC_table.tab

In [ ]:
!cat MPV_table.tab

### **Quality Control:**

Removing Duplicates

In [33]:
#Quality Control: Remove Duplicates
!/usr/bin/python3 /root/.local/bin/CollapseSeq.py -s M1-REV_primers-pass.fastq -n 20 --inner --uf R2Primer \
    --cf R1Primer --act set --outname M1

       START> CollapseSeq
        FILE> M1-REV_primers-pass.fastq
 MAX_MISSING> 20
 UNIQ_FIELDS> R1PRIMER
 COPY_FIELDS> R2PRIMER
COPY_ACTIONS> set
   MAX_FIELD> None
   MIN_FIELD> None
       INNER> True
KEEP_MISSING> False

PROGRESS> [0 missing] 18:36:46 |####################| 100% (34,709) 0.0 min

      OUTPUT> M1_collapse-unique.fastq
   SEQUENCES> 34709
      UNIQUE> 33788
   DUPLICATE> 921
UNDETERMINED> 0
         END> CollapseSeq



Creating Annotation Table

In [37]:
!/usr/bin/python3 /root/.local/bin/SplitSeq.py group -s M1_collapse-unique.fastq -f DUPCOUNT --num 2 --outname M1

    START> SplitSeq
  COMMAND> group
     FILE> M1_collapse-unique.fastq
    FIELD> DUPCOUNT
THRESHOLD> 2.0

PROGRESS> 18:39:30 |####################| 100% (33,788) 0.1 min

  OUTPUT1> M1_under-2.fastq
  OUTPUT2> M1_atleast-2.fastq
SEQUENCES> 33788
    PARTS> 2
      END> SplitSeq



In [35]:
!/usr/bin/python3 /root/.local/bin/ParseHeaders.py table -s M1_under-2.fastq -f ID DUPCOUNT R2Primer R1Primer

  START> ParseHeaders
COMMAND> table
   FILE> M1_under-2.fastq

PROGRESS> 18:37:14 |####################| 100% (33,156) 0.0 min

   OUTPUT> M1_under-2_headers.tab
SEQUENCES> 33156
     PASS> 33156
     FAIL> 0
      END> ParseHeaders



In [36]:
!cat M1_under-2_headers.tab

Streaming output truncated to the last 5000 lines.
M02058:623:000000000-DMLF3:1:1102:20498:20233	1	VHH-VecF1	short_hinge
M02058:623:000000000-DMLF3:1:1102:10821:20239	1	VHH-VecF1	short_hinge
M02058:623:000000000-DMLF3:1:1102:19503:20240	1	VHH-VecF1	short_hinge
M02058:623:000000000-DMLF3:1:1102:15685:20239	1	VHH-VecF1	short_hinge
M02058:623:000000000-DMLF3:1:1102:20541:20240	1	VHH-VecF1	short_hinge
M02058:623:000000000-DMLF3:1:1102:7936:20241	1	VHH-VecF1	short_hinge
M02058:623:000000000-DMLF3:1:1102:12215:20244	1	VHH-VecF1	short_hinge
M02058:623:000000000-DMLF3:1:1102:26534:20244	1	VHH-VecF1	short_hinge
M02058:623:000000000-DMLF3:1:1102:9127:20242	1	VHH-VecF1	short_hinge
M02058:623:000000000-DMLF3:1:1102:22967:20247	1	VHH-VecF1	short_hinge
M02058:623:000000000-DMLF3:1:1102:13387:20246	1	VHH-VecF1	short_hinge
M02058:623:000000000-DMLF3:1:1102:14432:20247	1	VHH-VecF1	short_hinge
M02058:623:000000000-DMLF3:1:1102:21324:20248	1	VHH-VecF1	short_hinge
M02058:623:000000000-DMLF3:1:1102:16054:2